#### Model Selection

In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, recall_score, precision_score, mean_squared_error,f1_score

In [2]:
# Loading the pickle file Customer_Loan

with open('Pickle_Files/Credit_Card.pkl', 'rb') as f1:
     df_train_cc= pickle.load(f1)
with open('Pickle_Files/40pct.pkl', 'rb') as f2:
     df_40= pickle.load(f2)

##### Train Test Split

In [3]:
# Split the dataset into training and test sets based on product
X_cc = df_train_cc.copy()
X_cc.drop(['Revenue_CC','Sale_CC'],inplace=True,axis=1)

y_sale_cc = df_train_cc['Sale_CC']

X_train_sl,X_test_sl,y_train_sl,y_test_sl = train_test_split(X_cc,y_sale_cc,test_size=0.2,random_state=0)

In [4]:
# Split the dataset into training and test sets based on revenue
X_cc = df_train_cc.copy()
X_cc.drop(['Revenue_CC','Sale_CC'],inplace=True,axis=1)

y_revenue_cc = df_train_cc['Revenue_CC']

X_train_rev,X_test_rev,y_train_rev,y_test_rev = train_test_split(X_cc,y_revenue_cc,test_size=0.2,random_state=0)

##### Normalize The Features

In [5]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_sl)
X_test = scaler.transform(X_test_sl)

#### Cross Validation

##### Classification

In [6]:
# Verifying which model has better score for Classification
model_param = {
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'param' : {'n_neighbors':[3,5,10,100],
                   'weights':['uniform','distance']}
        },
    'Logistic':{
        'model':LogisticRegression(max_iter=10000,solver='sag'), # converge warning hence sag
        'param': { 'C':[1,5,10]}
        },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'param' : {'n_estimators':[5,10,50,100]}
        },
    'DecisionTree':{
        'model': DecisionTreeClassifier(),
        'param' : {'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],  
            'min_samples_split': [2, 5, 10],  
            'min_samples_leaf': [1, 2, 4]}
        }
}


In [7]:
scores_class = []

for model_name, mp in model_param.items():
    hyperpara_model = GridSearchCV(mp['model'],mp['param'],cv=5,return_train_score=False)
    hyperpara_model.fit(X_cc,y_sale_cc)
    scores_class.append({
        'model': model_name,
        'best_score':hyperpara_model.best_score_,
        'best_param':hyperpara_model.best_params_
    })

In [8]:
scores_cc = pd.DataFrame(scores_class)
scores_cc

,model,best_score,best_param
0,KNeighbors,0.770883,"{'n_neighbors': 10, 'weights': 'uniform'}"
1,Logistic,0.730623,{'C': 1}
2,RandomForest,0.767796,{'n_estimators': 100}
3,DecisionTree,0.744031,"{'criterion': 'entropy', 'max_depth': 10, 'min..."


RandomForest seems to have the best score. Hence, I will be using this for model prediction. It is also more robust towards imbalanced datasets compared to KNeighbors + Logistic + DecisionTrees, as they are more biased towards majority data

In [9]:
rf = RandomForestClassifier(n_estimators=100)

rf.fit(X_train_sl,y_train_sl)

RandomForestClassifier()

In [10]:
y_pred_sl = rf.predict(X_test_sl)

In [11]:
# Probability of being correctly classified
y_pred_sl_prob=rf.predict_proba(X_test_sl)

##### Evaluation

In [12]:
confusion_matrix(y_pred_sl,y_test_sl)

array([[141,  36],
       [  9,   8]])

In [13]:
print('Evaluation metrics:')
print('Accuracy:', accuracy_score(y_test_sl,y_pred_sl))
print('Precision:', precision_score(y_test_sl, y_pred_sl))
print('Recall:', recall_score(y_test_sl, y_pred_sl))
print('F-1:', f1_score(y_test_sl, y_pred_sl))
print('AUC:', roc_auc_score(y_test_sl, y_pred_sl))

Evaluation metrics:
Accuracy: 0.7680412371134021
Precision: 0.47058823529411764
Recall: 0.18181818181818182
F-1: 0.26229508196721313
AUC: 0.5609090909090909


##### Regression

In [14]:
# # Define models
# models = {
#     'Linear': LinearRegression(),
#     'RandomForest': RandomForestRegressor(),
#     'DecisionTree': DecisionTreeRegressor(),
# }

# scores_rev = []

# for model_name, model in models.items():
#     # Use cross_val_score for simplicity
#     cv_scores = cross_val_score(model, X_cc, y_revenue_cc, cv=10)
#     scores_rev.append({
#         'model': model_name,
#         'mean_score': cv_scores.mean() * -1,
#         'std_score': cv_scores.std(),
        
#     })


In [15]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_colwidth', None)

# scores_rev = pd.DataFrame(scores_rev)
# scores_rev

In [16]:
model_param = {
    'Linear': {
        'model': LinearRegression(),
        'param' : {}
        },
    'RandomForest': {
        'model': RandomForestRegressor(),
        'param' : {'n_estimators':[5,10,50,100]}
        },
    'DecisionTree':{
        'model': DecisionTreeRegressor(),
        'param' : {'criterion': ['friedman_mse', 'squared_error', 'absolute_error', 'poisson'],
            'max_depth': [None, 10, 20, 30],  
            'min_samples_split': [2, 5, 10],  
            'min_samples_leaf': [1, 2, 4]}
        }
}

In [17]:
scores_regression = []

for model_name, mp in model_param.items():
    hyperpara_model = GridSearchCV(mp['model'],mp['param'],cv=5,return_train_score=False)
    hyperpara_model.fit(X_cc,y_sale_cc)
    scores_regression.append({
        'model': model_name,
        'best_score':hyperpara_model.best_score_,
        'best_param':hyperpara_model.best_params_
    })

In [18]:
scores_cl_reg = pd.DataFrame(scores_regression)
scores_cl_reg

,model,best_score,best_param
0,Linear,-0.013162,{}
1,RandomForest,0.030292,{'n_estimators': 100}
2,DecisionTree,-0.152666,"{'criterion': 'squared_error', 'max_depth': 10..."


Due to the void of hypertuning during cross validation, the RandomForest regression model is not adequate. Nonetheless, I will use the model as it is more robust towards imbalaced datasets.

In [19]:
rf_1= RandomForestRegressor(n_estimators=100)

rf_1.fit(X_train_rev,y_train_rev)

RandomForestRegressor()

In [20]:
y_pred_rev = rf_1.predict(X_test_rev)

##### Evaluation

In [21]:
mse=mean_squared_error(y_test_rev,y_pred_rev)
rmse=np.sqrt(mse)

print('MSE:', mse)
print('RMSE:', rmse)


MSE: 294.96320107262875
RMSE: 17.174492745715337


#### Feature Importance

In [22]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X_cc.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
feature_importances.head()

,importance
ActBal_CA,0.098951
ActBal_SA,0.080915
Tenure,0.067154
Age,0.065900
VolumeCred,0.059822


#### Targeting

In [23]:
df_40_cc = df_40.drop(['Count_CC','ActBal_CC','Revenue_CL','Sale_CL','Sale_CC','Sale_MF','Revenue_CC','Revenue_MF'], axis=1)

df_40_cc.shape

(646, 27)

In [24]:
scaler.transform(df_40_cc)

array([[1.        , 0.1375    , 0.099631  , ..., 0.04      , 0.        ,
        0.11764706],
       [1.        , 0.3375    , 0.64575646, ..., 0.08      , 0.        ,
        0.14705882],
       [0.        , 0.1125    , 0.25830258, ..., 0.16      , 0.05084746,
        0.05882353],
       ...,
       [1.        , 0.725     , 0.56457565, ..., 0.08      , 0.05084746,
        0.17647059],
       [1.        , 0.2625    , 0.33579336, ..., 0.04      , 0.01694915,
        0.26470588],
       [1.        , 0.6625    , 0.35793358, ..., 0.04      , 0.01694915,
        0.08823529]])

In [25]:
prob_cc = rf.predict_proba(df_40_cc)

In [26]:

rev_40=rf_1.predict(df_40_cc)

credit_card = pd.DataFrame({
    'Client': df_40_cc.index,
    'Revenue_CC':  rev_40,
    'Probability': prob_cc[:,1]
}).sort_values('Probability', ascending= False)

credit_card.rename(columns={'Revenue_CC':'Revenue'},inplace=True)
credit_card['Product']='CC'
credit_card.head(10)


,Client,Revenue,Probability,Product
52,19,4.955907,0.93,CC
443,104,3.861018,0.86,CC
32,1076,7.960554,0.85,CC
325,1262,3.283096,0.84,CC
560,1487,4.003871,0.84,CC
399,978,8.761864,0.84,CC
78,1227,3.338514,0.82,CC
301,587,5.374175,0.81,CC
242,389,7.795189,0.81,CC
122,701,4.789064,0.81,CC


In [27]:
file_path1 = 'Pickle_Files/Revenue_CC.pkl'

with open(file_path1, 'wb') as f1:
    pickle.dump(credit_card, f1)